In [20]:
#!/usr/bin/env python
from __future__ import print_function

from datetime import datetime
from collections import OrderedDict
import random
import os
import numpy as np
import pandas as pd
from re import A, S
from numpy import maximum
import math


import axioma
from axioma.assetset import AssetSet, ActionEntry
from axioma.account import Account
from axioma.accountgroup import AccountGroup
from axioma.workspace import DerbyProvider, Workspace
from axioma.workspace_element import ElementType
from axioma.costmodel import CostModel, CostStructure
from axioma.group import Group, Benchmark, Unit
from axioma.contentbuilder_group import ContentBuilderBenchmark, ContentBuilderGroup
from axioma.riskmodel import RiskModel
from axioma.strategy import Strategy, Objective, Target, Scope, MarketImpactType, PenaltyType
from axioma.rebalancing import Rebalancing, RebalancingStatus
from axioma.multiportfolio_rebalancing import MultiPortfolioRebalancing
from axioma.workspace_element import ElementType
from axioma.metagroup import Metagroup, DynamicMetagroup
from axioma.analytics import Analytics
import axioma.workspace_io as handler




In [21]:
axioma.ENDPOINT="http://localhost:8085/axioma-websrv"

In [ ]:




try:
    if 'ws' in locals() and ws is not None:
        try:
            ws.destroy()
        except Exception as e:
            # AxiomaError when server object is already gone; ignore that case
            if "ResourceNotFoundException" not in str(e):
                raise
finally:
    if 'ws' in locals():
        del ws



dates = ["2024-10-02","2024-10-03","2024-10-04"]

model = "WW51AxiomaSH"

# iterate through dates in dates list

############################## CREATE WORKSPACE
i=1
date = datetime.strptime(dates[i], '%Y-%m-%d').date()
next_period_date = datetime.strptime(dates[i+1], '%Y-%m-%d').date()
axioma_data_dir = r'C:/Users/jwbpa/Box/Altisma_Data/AxiomaDownloader-3.3.0-with-jre64/output/extractedFiles/database/${yyyy}/${mm}/'
db_provider = DerbyProvider(axioma_data_dir,
                            risk_models=model,
                            include_composites=True,
                            next_period_date=next_period_date,
                            returns_type="Gross Return")

ws = Workspace(f"BetaCompletion_Day1_WW51SH_{dates[i]}", date, data_provider=db_provider)

############################## LOAD DATA

ac_df = pd.read_csv(f"C:/Users/jwbpa/Box/Altisma_Data/Axioma/Example_Optimization/AlphaCapture_Optimized_{dates[i]}.csv", index_col="Name")
ac_account = Account(workspace = ws, 
        identity = "AC_Input", 
        holdings=ac_df.to_dict()["Shares"], 
        asset_map="Ticker Map")

ae_df = pd.read_csv(f"C:/Users/jwbpa/Box/Altisma_Data/Axioma/Example_Optimization/AlphaEngine_Combined/AlphaEngine_{dates[i-1]}.csv", index_col="Name")
ae_account = Account(workspace = ws, 
        identity = "AE_Input", 
        holdings=ae_df.to_dict()["Shares"], 
        asset_map="Ticker Map")

ac_final_df = pd.read_csv(f"C:/Users/jwbpa/Box/Altisma_Data/Axioma/Example_Optimization/AlphaCapture_Final/AlphaCapture_Final_{dates[i-1]}.csv", index_col="Name")
ac_final_account = Account(workspace = ws, 
        identity = "AC_Final_Input", 
        holdings=ac_final_df.to_dict()["Shares"], 
        asset_map="Ticker Map")


net_account=AccountGroup(workspace=ws, identity = "AE_AC_AGGREGATE")
net_account.add_account(ac_final_account)
net_account.add_account(ae_account)


net_long_value = net_account.get_long_value(price_group="Price", exclude_futures=False)
net_short_value = net_account.get_short_value(price_group="Price", exclude_futures=False)
net_cash_value = net_account.get_total_cash_value(price_group="Price")
net_gross_value=net_long_value+net_short_value+net_cash_value

ac_final_long_value = ac_final_account.get_long_value(price_group="Price", exclude_futures=False)
ac_final_short_value = ac_final_account.get_short_value(price_group="Price", exclude_futures=False)
ac_final_cash_value = ac_final_account.get_total_cash_value(price_group="Price")
ac_final_gross_value=ac_final_long_value+ac_final_short_value+ac_final_cash_value

ae_long_value = ae_account.get_long_value(price_group="Price", exclude_futures=False)
ae_short_value = ae_account.get_short_value(price_group="Price", exclude_futures=False)
ae_cash_value = ae_account.get_total_cash_value(price_group="Price")
ae_gross_value=ae_long_value+ae_short_value+ae_cash_value

allowed_cash_value=min(net_cash_value*1.5, 500000000)

# Benchmark
handler.load_assets_from_data_provider(workspace=ws, asset_names=["SPY"], asset_map="Ticker Map")
# SPY benchmark is referenced via its composition; explicit asset load is unnecessary
spy_benchmark = ContentBuilderBenchmark(workspace=ws, identity = "SPY_Benchmark", expression = "'Composition of 37P4NKR33'*1")
empty_benchmark = Benchmark(workspace = ws, identity = "empty benchmark", values={}, unit = Unit.Currency)

## Content Builder Attributes
# Content Builder for 60-Day MDV
Inv_60_Day_MDV = ContentBuilderGroup(workspace=ws, identity = "Inv_60_Day_MDV", expression = "(1/'60-Day MDV')")

# Content Builder for Account Currency
Account_Currency = ContentBuilderGroup(workspace=ws, identity = "Account_Currency", expression = "portfolioAsCurrency('AC_Input')*1")

In [ ]:
try:
    if 'strategy' in locals() and strategy is not None:
        try:
            strategy.destroy()
        except Exception as e:
            # AxiomaError when server object is already gone; ignore that case
            if "ResourceNotFoundException" not in str(e):
                raise
finally:
    if 'strategy' in locals():
        del strategy


############################## DEFINE STRATEGY
strategy = Strategy(workspace = ws, 
                    identity = "Strategy", 
                    allow_shorting=True, 
                    allow_crossover=False, 
                    allow_grandfathering=True,
                    enable_constraint_hierarchy=True)
strategy.set_local_universe(local_universe=list(ac_account.get_holdings().keys())+ list(ac_final_account.get_holdings().keys()) + ["CSH_USD__"])

############################## DEFINE OBJECTIVE TERMS
ar_term = axioma.strategy.create_risk_term(strategy = strategy, 
                                            identity = "activeRisk", 
                                            benchmark=Account_Currency,
                                            factor_weight=0, 
                                            risk_model=model,
                                            specific_weight=1.0,
                                            asset_set="MASTER",
                                            qualification=ac_account
                                            )
tc_term = axioma.strategy.create_market_impact_term(strategy = strategy, 
                                                    identity = "marketImpact",
                                                    buy_impact_group=Inv_60_Day_MDV,
                                                    sell_impact_group=Inv_60_Day_MDV,
                                                    market_impact_type=MarketImpactType.Quadratic
                                                    )
############################## DEFINE OBJECTIVE FUNCTION
terms = OrderedDict()
terms[ar_term] = 1.0
terms[tc_term] = 1.0
obj_fx = Objective(strategy = strategy, 
                    identity = "Objective", 
                    terms=terms, 
                    target=Target.Minimize, 
                    active=True)
############################## DEFINE CONSTRAINTS

# constraint 1
constraint_do_not_trade_ae=axioma.strategy.create_limit_trade_constraint(strategy = strategy,
                                                              identity = "constraintDoNotTradeAE",
                                                              minimum=0,
                                                              maximum=0,
                                                              scope=Scope.Asset,
                                                              unit=Unit.Percent
)
# add multiselection qualification based on ae_df
constraint_do_not_trade_ae.add_selection(element_type=ElementType.AssetSet, element="MASTER", qualification=ae_account)


# constraint 2
constraint_market=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintMarket",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

# add Market Intercept factor 
market_mgp = ws.get_metagroup(f"{model}.Market")
for group in market_mgp.get_groups():
    constraint_market.add_selection(element_type=ElementType.Group, element=group)



# constraint 3
constraint_country=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintCountry",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

country_mgp = ws.get_metagroup(f"{model}.Country")
for group in country_mgp.get_groups():
    constraint_country.add_selection(element_type=ElementType.Group, element=group)



# constraint 4
constraint_local=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintLocal",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

local_mgp = ws.get_metagroup(f"{model}.Local")
for group in local_mgp.get_groups():
    constraint_local.add_selection(element_type=ElementType.Group, element=group)



# constraint 5
constraint_currency=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintCurrency",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

currency_mgp = ws.get_metagroup(f"{model}.Currency")
for group in currency_mgp.get_groups():
    constraint_currency.add_selection(element_type=ElementType.Group, element=group)





# constraint 6
constraint_style=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintStyle",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )


style_mgp = ws.get_metagroup(f"{model}.Style")
for group in style_mgp.get_groups():
    constraint_style.add_selection(element_type=ElementType.Group, element=group)


# constraint 7
constraint_sector=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintSector",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

# add sector factors to multiselection except real estate and consumer discretionary
sectors = ws.get_metagroup(f"{model}.Sectors") 
sectors_metagroup_names_ex_real_estate_and_consumer_discretionary = [
    i.identity
    for i in sectors.get_metagroups()
    if i.identity != f"{model}.Real Estate-S" and i.identity != f"{model}.Consumer Discretionary-S"
]

for sector in sectors_metagroup_names_ex_real_estate_and_consumer_discretionary:
    constraint_sector.add_selection(element_type=ElementType.Metagroup, element=sector)








# constraint 8
constraint_realestate=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintRealestate",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

# add real estate  to multiselection
constraint_realestate.add_selection(element_type=ElementType.Metagroup, element=f"{model}.Real Estate-S")


# constraint 9
constraint_consumerdiscretionary=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "constraintConsumerdiscretionary",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

# add consumer discretionary to multiselection
constraint_consumerdiscretionary.add_selection(element_type=ElementType.Metagroup, element=f"{model}.Consumer Discretionary-S")






# constraint 8



sum_squared_weights=axioma.strategy.create_limit_model_deviation_constraint(strategy = strategy,
                                                                            identity = "sumSquaredWeights",
                                                                            maximum=25,
                                                                            penalty=4000,
                                                                            benchmark=empty_benchmark,
                                                                            penalty_type=PenaltyType.ScaledQuadratic,
                                                                            scope=Scope.Selection,
                                                                            unit=Unit.Percent
                                                                            )

# add NON-CASH ASSETS to selection
sum_squared_weights.add_selection(element_type=ElementType.AssetSet, element="NON-CASH ASSETS")


# constraint 9
cash_limit_holdings=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "cashLimitHoldings",
                                                                                 minimum=-allowed_cash_value,
                                                                                 maximum=allowed_cash_value,
                                                                                 scope=Scope.Asset,
                                                                                 unit=Unit.Currency
                                                                                 )
cash_limit_holdings.add_selection(element_type=ElementType.Asset, element="CSH_USD__")

# constraint 10
max_gross_exposure=axioma.strategy.create_limit_absolute_holding_constraint(strategy = strategy,
                                                                                 identity = "maxGrossExposure",
                                                                                 minimum=-200,
                                                                                 maximum=200,
                                                                                 scope=Scope.Aggregate,
                                                                                 unit=Unit.Percent
                                                                                 )
max_gross_exposure.add_selection(element_type=ElementType.AssetSet, element="MASTER")


############################## SET CONSTRAINT HIERARCHY CONSTRAINTS

strategy.set_constraint_hierarchy({
    "constraintLocal": 1,
    "constraintCountry": 2,
    "constraintCurrency":3
})

try:
    if 'rebal' in locals() and rebal is not None:
        try:
            rebal.destroy()
        except Exception as e:
            # AxiomaError when server object is already gone; ignore that case
            if "ResourceNotFoundException" not in str(e):
                raise
finally:
    if 'rebal' in locals():
        del rebal






############################## CREATE REBALANCING
rebal = MultiPortfolioRebalancing(workspace = ws, 
                    identity="BetaCompletion_Rebalancing", 
                    strategy=strategy
                    )

rebal.add_account(ac_final_account)
rebal.add_account(ae_account)

rebal.set_reference_size(net_gross_value)
#rebal.set_budget_size(1000000)
rebal.set_benchmark_size(net_gross_value)


rebal.set_rebalancing_defaults(round_lot_size=1,
                               use_cash_for_roundlot=True,
                               risk_model=model)



# === ONE-CELL: discover real constraint names -> ordered relaxation (minimal set) -> optional budget diagnostic ===
import re
from collections import OrderedDict

# Expect these to exist already:
try:
    _strat = strategy
    _rebal = rebal
except NameError as e:
    raise RuntimeError("This cell expects 'strategy' and 'rebal' to be defined before running.") from e

# ---------- status helpers (robust with/without enum import) ----------
def _ok_status(st):
    s = str(st)
    return ("SolutionFound" in s) or ("RelaxedSolutionFound" in s)

def _status_name(st):
    s = str(st)
    return s.split(".")[-1] if "." in s else s

# ---------- Your policy ----------
# Keep these HARD (never relax). Use your *real* identities:
NEVER_RELAX = {}  # 'BudgetConstraint' alias is not used in your notebook; real name is 'budget'

# Your preferred order (names may be fuzzy-resolved to actual identities discovered below)
PREFERRED_ORDER = [
    "sumSquaredWeights",
    "matchBenchmarkRealestateSectorExposure",
    "matchBenchmarkConsumerDiscretionarySectorExposure",
    "matchBenchmarkSectorExposure",
    "matchBenchmarkStyleExposure",
    "matchBenchmarkMarketExposure",
    "matchBenchmarkAggregateExposure", 
    "doNotTradeAE",
    'budget' # last resort
]

# Optional diagnostic to quantify budget mismatch if still infeasible with hard budget
ALLOW_TEMP_BUDGET_DIAG = False  # <-- set to True ONLY to measure how far budget would need to move (does NOT keep it relaxed in final solve)

# ---------- Make hierarchy strict & clear any prior hierarchy ----------
try:
    _strat.set_options(enable_constraint_hierarchy=True)
except Exception:
    pass

# Avoid reusing softened solutions; avoid auto micro-slack
try:
    _rebal.set_rebalancing_options(use_memento=False, hierarchy_tolerance_value=0.0)
except TypeError:
    try: _rebal.set_rebalancing_options(use_memento=False)
    except Exception: pass
    try: _rebal.set_rebalancing_options(hierarchy_tolerance_value=0.0)
    except Exception: pass
except Exception:
    pass

try:
    _strat.set_constraint_hierarchy({})
except Exception:
    pass

# ---------- Discovery: pull actual constraint objects and identities ----------
def _identity_from_obj(c):
    # Try attributes
    for attr in ("identity","name","id","_identity","_name"):
        if hasattr(c, attr):
            v = getattr(c, attr)
            if callable(v):
                try: v = v()
                except Exception: v = None
            if isinstance(v, str) and v.strip():
                return v.strip()
    # Try common methods
    for m in ("get_identity","get_name","getId"):
        if hasattr(c, m):
            try:
                v = getattr(c, m)()
                if isinstance(v, str) and v.strip():
                    return v.strip()
            except Exception:
                pass
    # Try to_dict
    if hasattr(c, "to_dict"):
        try:
            d = c.to_dict()
            for k in ("identity","name","id"):
                if k in d and isinstance(d[k], str) and d[k].strip():
                    return d[k].strip()
        except Exception:
            pass
    return None

def discover_constraints(strategy):
    discovered = []
    # 1) get_constraints()
    if hasattr(strategy, "get_constraints"):
        try:
            for c in (strategy.get_constraints() or []):
                nm = _identity_from_obj(c)
                discovered.append((nm, c.__class__.__name__))
        except Exception:
            pass
    # 2) fallback maps/attrs (very defensive; may duplicate)
    for meth in ("get_constraint_map","get_constraint_names","list_constraints"):
        if hasattr(strategy, meth):
            try:
                obj = getattr(strategy, meth)()
                if isinstance(obj, dict):
                    for k, v in obj.items():
                        name = str(k) if k is not None else None
                        if name: discovered.append((name, getattr(v, "__class__", type(v)).__name__))
                elif isinstance(obj, (list, tuple, set)):
                    for x in obj:
                        if isinstance(x, str):
                            discovered.append((x, ""))
                        else:
                            nm = _identity_from_obj(x)
                            discovered.append((nm, getattr(x, "__class__", type(x)).__name__))
            except Exception:
                pass

    # Clean
    clean = []
    for nm, tp in discovered:
        if nm and isinstance(nm, str):
            clean.append((nm, tp))
    # Deduplicate by name while preferring the first seen type
    seen, rows = set(), []
    for nm, tp in clean:
        if nm not in seen:
            rows.append((nm, tp))
            seen.add(nm)
    return rows

rows = discover_constraints(_strat)

print("\n=== DISCOVERED CONSTRAINT IDENTITIES (from strategy) ===")
if not rows:
    print("(None discovered; API may hide them on this object. Continue anyway.)")
else:
    w = max(len(r[0]) for r in rows)
    for nm, tp in rows:
        print(f"  {nm.ljust(w)}  [{tp}]")

DISCOVERED_NAMES = [nm for nm,_ in rows]

# ---------- Resolve your preferred names to real identities (lenient match) ----------
def _norm(s): return re.sub(r"[^a-z0-9]+", "", s.lower()) if isinstance(s, str) else ""
disc_norm = { _norm(nm): nm for nm in DISCOVERED_NAMES }

resolved = []
unresolved = []
for wanted in PREFERRED_ORDER:
    if wanted in DISCOVERED_NAMES:
        resolved.append(wanted)
        continue
    wn = _norm(wanted)
    match = disc_norm.get(wn)
    if not match:
        # lenient: substring either way
        for dn_raw in DISCOVERED_NAMES:
            dn = _norm(dn_raw)
            if wn and (wn in dn or dn in wn):
                match = dn_raw
                break
    if match:
        resolved.append(match)
    else:
        unresolved.append(wanted)

# Remove hard set and duplicates from resolved
RESOLVED_ORDER = []
seen = set()
for nm in resolved:
    if nm in NEVER_RELAX:  # never relax
        continue
    if nm not in seen:
        RESOLVED_ORDER.append(nm); seen.add(nm)

print("\nResolved preferred names to real identities (order):", RESOLVED_ORDER)
if unresolved:
    print("Could not resolve (check your naming):", unresolved)

# ---------- Helpers for hierarchy/solve/report ----------
def _apply_hierarchy(names_in_order):
    mapping = OrderedDict((n, i+1) for i, n in enumerate(names_in_order))
    _strat.set_constraint_hierarchy(mapping)

def _solve():
    return _rebal.solve()

def _ordered_relax_minimal(order_names):
    # Start with nothing relaxable
    try: _apply_hierarchy([])
    except Exception: pass

    relaxed, sol = [], None

    # Add one-by-one
    for nm in order_names:
        relaxed.append(nm)
        _apply_hierarchy(relaxed)
        try:
            s_try = _solve()
            if _ok_status(s_try.get_status()):
                sol = s_try
                break
        except Exception:
            sol = None
            continue

    # If still none, try all at once (in this order)
    if sol is None and order_names:
        _apply_hierarchy(order_names)
        try:
            s_try = _solve()
            if _ok_status(s_try.get_status()):
                sol, relaxed = s_try, list(order_names)
            else:
                return None, []
        except Exception:
            return None, []

    # Reverse-harden to minimal
    for nm in list(relaxed)[::-1]:
        trial = [x for x in relaxed if x != nm]
        _apply_hierarchy(trial)
        try:
            s2 = _solve()
            if _ok_status(s2.get_status()):
                sol, relaxed = s2, trial
            else:
                _apply_hierarchy(relaxed)  # restore
        except Exception:
            _apply_hierarchy(relaxed)      # restore

    return sol, relaxed

def _report_hard_violations(solution, focus=None, top_per_constraint=5, base_order=None):
    try:
        rows = solution.get_constraint_values()
    except Exception as e:
        print("Constraint values unavailable:", e)
        return
    if focus:
        fset = {n.lower() for n in focus}
        rows = [r for r in rows if str(r.get("constraint_name","")).lower() in fset]
    by = {}
    for r in rows:
        by.setdefault(r.get("constraint_name"), []).append(r)

    def _order_key(k):
        if base_order and k in base_order: return base_order.index(k)
        return 10**9

    print("\n=== PER-CONSTRAINT hard_violation (vs ORIGINAL hard bounds) ===")
    if not by:
        print("(No rows returned for the requested constraints.)")
        return

    for cname in sorted(by.keys(), key=_order_key):
        items = by[cname]
        worst = max(items, key=lambda d: abs(d.get("hard_violation") or 0.0))
        unit  = worst.get("constraint_unit")
        max_hv = abs(worst.get("hard_violation") or 0.0)
        print(f"\n[{cname}]  max hard_violation={max_hv:.6g} {unit or ''}")
        for r in sorted(items, key=lambda d: abs(d.get("hard_violation") or 0.0), reverse=True)[:top_per_constraint]:
            print(
                f"  • {r.get('selection_name')}: "
                f"final={r.get('final_value')}  "
                f"applied=({r.get('applied_min')},{r.get('applied_max')})  "
                f"violation={r.get('violation')}  "
                f"hard_violation={r.get('hard_violation')} {unit or ''}"
            )
# === PATCH: robust "hard-violation" reporter for Axioma ConstraintValue objects OR dicts ===
import re

def _normalize_key(k: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", (k or "").lower())

def _safe_call(v):
    try:
        return v() if callable(v) else v
    except Exception:
        return v

def _lookup_attr(row, candidates):
    """
    Try several attribute/method spellings on an object or dict, then do a
    case/underscore-insensitive scan of available attributes as a last resort.
    """
    # 1) direct hit on dict
    if isinstance(row, dict):
        for c in candidates:
            if c in row:
                return row[c]
        # also try normalized match
        norm_map = {_normalize_key(k): k for k in row.keys()}
        for c in candidates:
            nk = _normalize_key(c)
            if nk in norm_map:
                return row[norm_map[nk]]
        return None

    # 2) direct attribute/method on object
    for c in candidates:
        if hasattr(row, c):
            return _safe_call(getattr(row, c))

    # 3) "getXxx" method variants
    for c in candidates:
        cc = c[0].upper() + c[1:] if c else c
        for alt in (f"get_{c}", f"get{cc}"):
            if hasattr(row, alt):
                return _safe_call(getattr(row, alt))

    # 4) normalized attribute name search
    try:
        names = dir(row)
        norm_map = {_normalize_key(n): n for n in names}
        for c in candidates:
            nk = _normalize_key(c)
            if nk in norm_map:
                return _safe_call(getattr(row, norm_map[nk]))
    except Exception:
        pass

    # 5) to_dict / toDict fallback
    for m in ("to_dict", "toDict"):
        if hasattr(row, m):
            try:
                d = getattr(row, m)()
                if isinstance(d, dict):
                    return _lookup_attr(d, candidates)
            except Exception:
                pass

    return None

def _row_to_dict(row):
    """
    Normalize a single row (dict OR object) to a dict with consistent keys.
    """
    fields = {
        "constraint_name": ["constraint_name","constraintName","name","constraint","constraint_id","constraintId"],
        "selection_name":  ["selection_name","selectionName","selection","bucket","asset","group","groupName"],
        "final_value":     ["final_value","finalValue","value","final","actualValue","measureValue"],
        "applied_min":     ["applied_min","appliedMin","min","appliedLower","lower","lowerBound","appliedLowerBound"],
        "applied_max":     ["applied_max","appliedMax","max","appliedUpper","upper","upperBound","appliedUpperBound"],
        "violation":       ["violation","applied_violation","appliedViolation","slack"],
        "hard_violation":  ["hard_violation","hardViolation","hardSlack","violation_vs_original","violationVsOriginal"],
        "constraint_unit": ["constraint_unit","constraintUnit","unit","units"],
        "side":            ["side"]
    }
    out = {}
    for k, cands in fields.items():
        out[k] = _lookup_attr(row, cands)
    return out

def _report_hard_violations(solution, focus=None, top_per_constraint=5, base_order=None):
    """
    Print compact per-constraint report of 'hard_violation' (vs ORIGINAL hard bounds),
    robust to Axioma returning dicts OR 'ConstraintValue' objects.
    """
    # 1) fetch rows
    rows = []
    try:
        rows = solution.get_constraint_values()
    except Exception as e:
        print("get_constraint_values() failed:", e)
        rows = []

    # 2) normalize rows to dicts
    norm = [_row_to_dict(r) for r in (rows or [])]
    norm = [r for r in norm if r.get("constraint_name")]

    # 3) optional focus filter
    if focus:
        fset = {str(n).lower() for n in focus}
        norm = [r for r in norm if str(r.get("constraint_name","")).lower() in fset]

    # 4) group by constraint
    by = {}
    for r in norm:
        by.setdefault(r["constraint_name"], []).append(r)

    # 5) nothing to show?
    print("\n=== PER-CONSTRAINT hard_violation (vs ORIGINAL hard bounds) ===")
    if not by:
        print("(No rows matched; verify constraint names in 'focus'.)")
        # show quick debug if we can
        sample = rows[0] if rows else None
        if sample is not None:
            print("Example row type:", type(sample))
            try:
                some = dir(sample)
                print("Example available attributes (head):",
                      [a for a in some if not a.startswith("_")][:25])
            except Exception:
                pass
        return by

    # 6) printing order helper
    if base_order is None:
        base_order = []
    def _order_key(name):
        try:
            return list(base_order).index(name)
        except ValueError:
            return 10**9

    # 7) emit report
    def _absfloat(x):
        try:
            return abs(float(x))
        except Exception:
            return 0.0

    for cname in sorted(by.keys(), key=_order_key):
        items = by[cname]
        worst = max(items, key=lambda d: _absfloat(d.get("hard_violation")))
        unit  = worst.get("constraint_unit") or ""
        print(f"\n[{cname}]  max hard_violation={_absfloat(worst.get('hard_violation')):.6g} {unit}")
        # Top offenders within the constraint
        top = sorted(items, key=lambda d: _absfloat(d.get("hard_violation")), reverse=True)[:top_per_constraint]
        for r in top:
            print("  • {sel}: final={fv}  applied=({mn},{mx})  violation={v}  hard_violation={hv} {u}".format(
                sel=r.get("selection_name"),
                fv=r.get("final_value"),
                mn=r.get("applied_min"),
                mx=r.get("applied_max"),
                v=r.get("violation"),
                hv=r.get("hard_violation"),
                u=unit
            ))
    return by





# ---------- RUN: your resolved order ----------
sol, relaxed = _ordered_relax_minimal(RESOLVED_ORDER)

# ---------- IF STILL infeasible: try adding any other discovered constraints (except hard set) ----------
if sol is None:
    extras = [nm for nm in DISCOVERED_NAMES if nm not in NEVER_RELAX and nm not in RESOLVED_ORDER]
    if extras:
        print("\nExpanding relax order with discovered constraints (excluding hard set):", extras)
        sol, relaxed = _ordered_relax_minimal(RESOLVED_ORDER + extras)

# ---------- Final report or diagnostic ----------
if sol is None:
    print("\n=== RESULT: Still infeasible even after relaxing resolved & discovered constraints (except hard set). ===")
    print("Likely a hard conflict (often budget vs doNotTradeAE or another hard limit like an exact cardinality).")
    # Optional diagnostic: temporarily allow budget to relax JUST to measure mismatch
    if ALLOW_TEMP_BUDGET_DIAG and ("budget" in DISCOVERED_NAMES):
        print("\n--- DIAGNOSTIC (temporary): allowing 'budget' to relax to measure mismatch ---")
        diag_order = ["budget"] + [nm for nm in (RESOLVED_ORDER + extras if 'extras' in locals() else RESOLVED_ORDER) if nm != "budget"]
        _apply_hierarchy(diag_order)
        try:
            s_diag = _solve()
            if _ok_status(s_diag.get_status()):
                print("Diagnostic solve succeeded only when 'budget' is relaxable. Here's how far it needed to move:")
                _report_hard_violations(s_diag, focus={"budget"})
                print("\n(End of diagnostic. Re-run with ALLOW_TEMP_BUDGET_DIAG=False for normal behavior.)")
            else:
                print("Diagnostic solve still infeasible even with 'budget' relaxable.")
        except Exception as e:
            print("Diagnostic solve failed:", e)
else:
    print("\n=== SUMMARY ===")
    print("Final status:", _status_name(sol.get_status()))
    print("Relaxed (minimal set):", relaxed)
    print("Hard (explicit):", sorted(NEVER_RELAX))
    # === Call it (same way you did before). If you don't have PREFERRED_ORDER defined, pass list(relaxed).
    _report_hard_violations(sol, focus=set(relaxed), base_order=PREFERRED_ORDER if "PREFERRED_ORDER" in globals() else list(relaxed))


    # Optional: trades (safe only when solved)
    try:
        try:
            trades = sol.get_trade_list(include_initial_holdings=False,
                                        include_multi_rebal_settings=False,
                                        sales_are_negative=True,
                                        split_crossover_trades=True)
        except TypeError:
            trades = sol.get_trade_list(include_initial_holdings=False,
                                        include_rebal_settings=False,
                                        sales_are_negative=True,
                                        split_crossover_trades=True)
        print("\nTrade list available. Keys:", list(trades)[:10])
    except Exception as e:
        print("Trades unavailable:", e)


In [ ]:
############################## SOLVE REBALANCING
sol = rebal.solve()
print(sol.get_status())
if sol.get_status()==RebalancingStatus.SolutionFound or sol.get_status()==RebalancingStatus.RelaxedSolutionFound:
    fh = sol.get_final_holdings(asset_map = "Ticker Map")
#sol.get_final_holdings(asset_map=None)
ae_fh=sol.get_final_account_holdings_for_account(ae_account, asset_map="Ticker Map")
ac_fh=sol.get_final_account_holdings_for_account(ac_final_account, asset_map="Ticker Map")

RebalancingStatus.RelaxedSolutionFound


In [26]:
# === DETAILED VIEW OF ac_fh (AC Account Final Holdings) ===

print("=" * 80)
print("AC_FH DETAILED INFORMATION")
print("=" * 80)

# Extract holdings using Axioma Holdings API methods
holdings_dict = {}
try:
    # Try get_holding_lots() first - this likely returns shares
    holding_lots = ac_fh.get_holding_lots()
    if holding_lots:
        # If it's a dict, use it directly (key=asset, value=shares)
        if isinstance(holding_lots, dict):
            holdings_dict = holding_lots
        # If it's a list/tuple, convert to dict
        elif isinstance(holding_lots, (list, tuple)):
            holdings_dict = {item.get('asset', item.get('name', f'asset_{i}')): 
                            item.get('shares', item.get('quantity', item.get('value', 0)))
                            for i, item in enumerate(holding_lots)}
        else:
            # Try to convert to dict if iterable
            try:
                holdings_dict = dict(holding_lots)
            except:
                pass
except Exception as e:
    print(f"Note: get_holding_lots() result: {type(holding_lots) if 'holding_lots' in locals() else 'N/A'}")
    if 'holding_lots' in locals() and holding_lots:
        print(f"  Sample data: {list(holding_lots)[:3] if hasattr(holding_lots, '__iter__') else holding_lots}")

# If still empty, try get_asset_values()
if not holdings_dict:
    try:
        asset_values = ac_fh.get_asset_values()
        if asset_values:
            if isinstance(asset_values, dict):
                holdings_dict = asset_values
            else:
                try:
                    holdings_dict = dict(asset_values)
                except:
                    pass
    except Exception as e:
        pass

# If still empty, try accessing internal attributes
if not holdings_dict:
    try:
        # Try accessing _holding_lots or _asset_values directly
        if hasattr(ac_fh, '_holding_lots'):
            lots = ac_fh._holding_lots
            if isinstance(lots, dict):
                holdings_dict = lots
            elif isinstance(lots, (list, tuple)):
                # Extract asset and shares from lot objects
                for i, lot in enumerate(lots):
                    if hasattr(lot, 'asset') and hasattr(lot, 'shares'):
                        holdings_dict[lot.asset] = lot.shares
                    elif isinstance(lot, dict):
                        holdings_dict[lot.get('asset', f'asset_{i}')] = lot.get('shares', lot.get('quantity', 0))
        elif hasattr(ac_fh, '_asset_values'):
            holdings_dict = ac_fh._asset_values if isinstance(ac_fh._asset_values, dict) else {}
    except Exception as e:
        pass

print(f"Extracted {len(holdings_dict)} holdings")
if not holdings_dict:
    print(f"\nDebug info:")
    print(f"  ac_fh type: {type(ac_fh)}")
    print(f"  ac_fh methods: {[attr for attr in dir(ac_fh) if not attr.startswith('_') and callable(getattr(ac_fh, attr, None))]}")

if holdings_dict:
    # Convert to DataFrame
    ac_fh_df = pd.DataFrame.from_dict(holdings_dict, orient='index', columns=['Shares'])
    ac_fh_df.index.name = 'Ticker'
    ac_fh_df = ac_fh_df.reset_index()
    ac_fh_df = ac_fh_df.sort_values('Shares', ascending=False, key=abs)
    
    # Get prices for value calculation
    try:
        price_group = ws.get_group("Price")
        prices_dict = {}
        for ticker in ac_fh_df['Ticker']:
            try:
                if hasattr(price_group, 'get_composition_asset'):
                    prices_dict[ticker] = float(price_group.get_composition_asset(ticker))
                elif hasattr(price_group, 'get_composition'):
                    comp = price_group.get_composition()
                    if isinstance(comp, dict):
                        prices_dict[ticker] = float(comp.get(ticker, 0))
            except Exception:
                prices_dict[ticker] = np.nan
    except Exception as e:
        print(f"Warning: Could not retrieve prices: {e}")
        prices_dict = {}
    
    # Add price and value columns
    ac_fh_df['Price'] = ac_fh_df['Ticker'].map(prices_dict)
    ac_fh_df['Market_Value'] = ac_fh_df['Shares'] * ac_fh_df['Price']
    
    # Summary statistics
    print("\n--- SUMMARY STATISTICS ---")
    total_positions = len(ac_fh_df)
    long_positions = len(ac_fh_df[ac_fh_df['Shares'] > 0])
    short_positions = len(ac_fh_df[ac_fh_df['Shares'] < 0])
    zero_positions = len(ac_fh_df[ac_fh_df['Shares'] == 0])
    
    total_shares_long = ac_fh_df[ac_fh_df['Shares'] > 0]['Shares'].sum()
    total_shares_short = abs(ac_fh_df[ac_fh_df['Shares'] < 0]['Shares'].sum())
    total_shares_net = ac_fh_df['Shares'].sum()
    
    
    print(f"Total Positions: {total_positions}")
    print(f"  - Long positions: {long_positions}")
    print(f"  - Short positions: {short_positions}")
    print(f"  - Zero positions: {zero_positions}")
    print(f"\nTotal Shares:")
    print(f"  - Long: {total_shares_long:,.2f}")
    print(f"  - Short: {total_shares_short:,.2f}")
    print(f"  - Net: {total_shares_net:,.2f}")

    
    # Display holdings DataFrame
    print("\n--- ALL HOLDINGS (sorted by absolute shares) ---")
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_columns', None)
    display(ac_fh_df.style.format({
        'Shares': '{:,.6f}',
        'Price': '{:,.2f}',
        'Market_Value': '${:,.2f}'
    }))
    
    
    
    
    
    # Reset display options
    pd.reset_option('display.max_rows')
    pd.reset_option('display.width')
    pd.reset_option('display.max_columns')
else:
    print("No holdings data available or could not convert holdings to dictionary format.")
    print(f"\nac_fh object details:")
    print(f"  Type: {type(ac_fh)}")
    print(f"  String representation: {str(ac_fh)}")
    if hasattr(ac_fh, '__dict__'):
        print(f"  Attributes: {list(ac_fh.__dict__.keys())}")


AC_FH DETAILED INFORMATION
Extracted 166 holdings

--- SUMMARY STATISTICS ---
Total Positions: 166
  - Long positions: 78
  - Short positions: 88
  - Zero positions: 0

Total Shares:
  - Long: 18,924,811.00
  - Short: 7,778,076.00
  - Net: 11,146,735.00

--- ALL HOLDINGS (sorted by absolute shares) ---


,Ticker,Shares,Price,Market_Value
64,CSH_USD,"3,232,330.000000",nan,$nan
77,ERJ,"1,220,065.000000",nan,$nan
133,WMT,"1,121,304.000000",nan,$nan
53,HPE,"1,111,785.000000",nan,$nan
32,CYH,"-1,084,002.000000",nan,$nan
70,PLTR,"1,044,561.000000",nan,$nan
71,KMI,"915,919.000000",nan,$nan
104,GOOGL,"847,287.000000",nan,$nan
158,WTTR,"727,761.000000",nan,$nan
85,WBD,"-713,159.000000",nan,$nan


In [ ]:
# === SAVE ac_fh TO CSV (Name and Shares columns only) ===

# Extract holdings from ac_fh Holdings object
holdings_dict = {}
try:
    # Try get_holding_lots() first - returns shares
    holding_lots = ac_fh.get_holding_lots()
    if holding_lots:
        if isinstance(holding_lots, dict):
            holdings_dict = holding_lots
        elif isinstance(holding_lots, (list, tuple)):
            # Convert list of lots to dict (sum shares if multiple lots per asset)
            for lot in holding_lots:
                if isinstance(lot, dict):
                    asset = lot.get('asset', lot.get('name'))
                    shares = lot.get('shares', lot.get('quantity', 0))
                else:
                    # Try attribute access
                    asset = getattr(lot, 'asset', getattr(lot, 'name', None))
                    shares = getattr(lot, 'shares', getattr(lot, 'quantity', 0))
                
                if asset:
                    if asset in holdings_dict:
                        holdings_dict[asset] += shares
                    else:
                        holdings_dict[asset] = shares
        else:
            try:
                holdings_dict = dict(holding_lots)
            except:
                pass
except Exception as e:
    print(f"Note: get_holding_lots() error: {e}")

# If still empty, try get_asset_values() and convert to shares
if not holdings_dict:
    try:
        asset_values = ac_fh.get_asset_values()
        if asset_values:
            if isinstance(asset_values, dict):
                # If values are provided, we need prices to convert to shares
                # For now, assume values might actually be shares (check magnitude)
                holdings_dict = asset_values
    except:
        pass

# If still empty, try internal attributes
if not holdings_dict:
    try:
        if hasattr(ac_fh, '_holding_lots'):
            lots = ac_fh._holding_lots
            if isinstance(lots, dict):
                holdings_dict = lots
            elif isinstance(lots, (list, tuple)):
                for lot in lots:
                    if hasattr(lot, 'asset') and hasattr(lot, 'shares'):
                        asset = lot.asset
                        shares = lot.shares
                        if asset in holdings_dict:
                            holdings_dict[asset] += shares
                        else:
                            holdings_dict[asset] = shares
                    elif isinstance(lot, dict):
                        asset = lot.get('asset', lot.get('name'))
                        shares = lot.get('shares', lot.get('quantity', 0))
                        if asset:
                            if asset in holdings_dict:
                                holdings_dict[asset] += shares
                            else:
                                holdings_dict[asset] = shares
    except:
        pass

if holdings_dict:
    # Create DataFrame with Name and Shares columns
    ac_fh_df = pd.DataFrame([
        {'Name': asset, 'Shares': shares}
        for asset, shares in holdings_dict.items()
    ])
    
    # Sort by absolute shares (descending)
    ac_fh_df = ac_fh_df.reindex(ac_fh_df['Shares'].abs().sort_values(ascending=False).index)
    ac_fh_df = ac_fh_df.reset_index(drop=True)
    
    # Save to CSV
    output_file = f"C:/Users/jwbpa/Box/Altisma_Data/Axioma/Example_Optimization/AlphaCapture_Final/AlphaCapture_Final_{dates[i]}.csv"
    ac_fh_df.to_csv(output_file, index=False)
    print(f"Saved {len(ac_fh_df)} holdings to {output_file}")
    print(f"\nFirst few rows:")
    print(ac_fh_df.head(10).to_string(index=False))
else:
    print("Error: Could not extract holdings from ac_fh")
    print(f"Debug: ac_fh type = {type(ac_fh)}")


Saved 166 holdings to C:/Users/jwbpa/Box/Altisma_Data/Axioma/Example_Optimization/AlphaCapture_Final/AlphaCapture_Final_2024-10-02.csv

First few rows:
   Name     Shares
CSH_USD  3232330.0
    ERJ  1220065.0
    WMT  1121304.0
    HPE  1111785.0
    CYH -1084002.0
   PLTR  1044561.0
    KMI   915919.0
  GOOGL   847287.0
   WTTR   727761.0
    WBD  -713159.0


In [29]:

RM  = model
BM  = spy_benchmark
REF = float(rebal.get_reference_size())


# 1) Build Analytics (use workspace defaults + explicit args we care about)
analyzer = Analytics(
    workspace=ws,
    price_group="Price",        # <-- change if your price group id differs
    risk_model=RM,
    benchmark=BM,
    alphas="Period Return",     # we'll use this for realized daily attribution
    reference_value=REF,
    asset_map="Ticker Map"      # keep if you rely on mapping to tickers
)


# ---------- helpers ----------
def _try_call(method_name, candidates):
    """Try calling analyzer.<method_name> with each kwarg dict in 'candidates' until one works."""
    fn = getattr(analyzer, method_name)
    last_err = None
    for kwargs in candidates:
        try:
            return fn(**{k: v for k, v in kwargs.items() if v is not None})
        except TypeError as e:
            last_err = e;  continue
        except Exception as e:
            last_err = e;  continue
    if last_err is not None:
        raise last_err
    raise RuntimeError(f"No candidates provided for {method_name}")

def _sum_numeric_leaves(obj):
    """Sum numeric leaves in nested dict/list structures."""
    import numbers
    total = 0.0
    stack = [obj]
    while stack:
        x = stack.pop()
        if isinstance(x, dict):
            stack.extend(x.values())
        elif isinstance(x, (list, tuple)):
            stack.extend(x)
        elif isinstance(x, numbers.Number):
            total += float(x)
    return float(total)

def _pct_of_ref(x): return (float(x) / REF * 100.0) if REF else float("nan")

# ---------- active holdings & exposures ----------
ah = analyzer.compute_active_holdings(holdings=fh, benchmark=BM, reference_value=REF)

ah_exposures = (
    pd.DataFrame.from_dict({"Exposure": analyzer.compute_factor_exposures(risk_model=RM, holdings=ah)}) / REF
)
ah_exposures.index.name = "Factor"
ah_exposures.sort_values("Exposure", ascending=False, inplace=True)

# ---------- prices (for RC fallback) ----------
price_group = ws.get_group("Price")
prices = price_group.get_composition() if hasattr(price_group, "get_composition") else {}
if not prices:
    # minimal fallback: ask per-asset if broadcast composition isn't available
    assets = set(fh.keys()) | set(ah.keys())
    prices = {a: float(price_group.get_composition_asset(a)) for a in assets}

# ---------- Tracking error (total) ----------
try:
    # Preferred: explicit active risk call
    te_total_ccy = float(_try_call("compute_active_total_risk", [
        dict(risk_model=RM, holdings=fh, benchmark=BM, reference_value=REF),
        dict(holdings=fh, benchmark=BM, reference_value=REF),
        dict(risk_model=RM, holdings=fh, benchmark=BM),
        dict(holdings=fh, benchmark=BM),
    ]))
except Exception:
    # Fallback: total risk of ACTIVE holdings (= TE)
    te_total_ccy = float(_try_call("compute_total_risk", [
        dict(risk_model=RM, holdings=ah),
        dict(holdings=ah),
    ]))

# ---------- Factor risk (active) ----------
te_factor_ccy, factor_var_map = None, None
# Best: factor variance decomposition of ACTIVE risk, then sqrt(sum)
try:
    factor_var_map = _try_call("compute_active_factor_variance_decomposition", [
        dict(risk_model=RM, holdings=fh, benchmark=BM, reference_value=REF),
        dict(risk_model=RM, holdings=ah),
        dict(holdings=ah),
    ])
    if isinstance(factor_var_map, dict) and factor_var_map:
        te_factor_ccy = math.sqrt(max(0.0, _sum_numeric_leaves(factor_var_map)))
except Exception:
    factor_var_map = None

# If variance decomp not available, try direct factor risk on ACTIVE holdings
if te_factor_ccy is None:
    try:
        te_factor_ccy = float(_try_call("compute_factors_risk", [
            dict(risk_model=RM, holdings=ah),   # <-- no benchmark/reference_value here
            dict(holdings=ah),
        ]))
    except Exception:
        te_factor_ccy = float("nan")

# ---------- Specific risk (active) ----------
# Prefer a direct call if your build has it; otherwise derive from variance identity
try:
    te_spec_ccy = float(_try_call("compute_active_specific_risk", [
        dict(risk_model=RM, holdings=fh, benchmark=BM, reference_value=REF),
        dict(holdings=fh, benchmark=BM, reference_value=REF),
    ]))
except Exception:
    try:
        # Specific of ACTIVE holdings equals active specific
        te_spec_ccy = float(_try_call("compute_specific_risk", [
            dict(risk_model=RM, holdings=ah),
            dict(holdings=ah),
        ]))
    except Exception:
        # Last resort: TE^2 - Factor^2
        if te_factor_ccy == te_factor_ccy:  # not NaN
            te_spec_ccy = math.sqrt(max(0.0, te_total_ccy**2 - te_factor_ccy**2))
        else:
            te_spec_ccy = float("nan")

# ---------- Risk summary table ----------
risk_summary = pd.DataFrame(
    {
        "Risk (ccy)": [te_total_ccy, te_factor_ccy, te_spec_ccy],
        "Risk (%)"  : [_pct_of_ref(te_total_ccy), _pct_of_ref(te_factor_ccy), _pct_of_ref(te_spec_ccy)],
    },
    index=["Tracking Error (Total)", "Active Factor Risk", "Active Specific Risk"],
)

# ---------- Factor variance table (if available) ----------
factor_var_df = None
if isinstance(factor_var_map, dict) and factor_var_map:
    fv = {str(k): (_sum_numeric_leaves(v) if isinstance(v, (dict, list, tuple)) else float(v))
          for k, v in factor_var_map.items()}
    fv = pd.Series(fv, name="Variance (ccy^2)").sort_values(ascending=False)
    total_var = te_total_ccy**2 if te_total_ccy == te_total_ccy else float("nan")
    factor_var_df = pd.DataFrame({
        "Variance (ccy^2)": fv,
        "% of TE variance": (fv / total_var * 100.0) if total_var and total_var == total_var else np.nan
    })

# ---------- Asset-level % contribution to TE ----------
risk_contrib_df = None
# Prefer a direct API if present
try:
    arc_map = _try_call("compute_asset_risk_contribution", [
        dict(risk_model=RM, holdings=fh, benchmark=BM, benchmark_value=REF),
        dict(risk_model=RM, holdings=fh, benchmark=BM, reference_value=REF),
        dict(risk_model=RM, holdings=ah),
        dict(holdings=ah),
    ])
    rc = pd.Series(arc_map, dtype=float).sort_values(ascending=False)
    # If values look like fractions, convert to %
    if rc.abs().max() <= 1.0:
        rc = rc * 100.0
    risk_contrib_df = rc.rename("% of TE").to_frame()
except Exception:
    # Fallback via MCAR: RC% = (active_value * MCAR) / TE * 100
    try:
        mcar_map = _try_call("compute_mcar", [
            dict(risk_model=RM, holdings=ah),
            dict(holdings=ah),
        ])
        ah_s = pd.Series(ah, dtype=float)
        px_s = pd.Series({a: prices.get(a, np.nan) for a in ah_s.index}, dtype=float)
        active_value = ah_s * px_s
        mcar_s = pd.Series(mcar_map, dtype=float).reindex(active_value.index).fillna(0.0)
        rc_ccy = active_value * mcar_s
        risk_contrib_df = (rc_ccy / te_total_ccy * 100.0).sort_values(ascending=False).rename("% of TE").to_frame() if te_total_ccy else pd.DataFrame()
    except Exception:
        risk_contrib_df = pd.DataFrame(columns=["% of TE"])

# ---------- Active total / factor / specific return (daily) ----------
active_total_return_ccy = float(_try_call("compute_expected_return", [
    dict(holdings=ah, alphas="Period Return"),
    dict(holdings=ah, alphas=ws.get_group("Period Return")),
]))

# Factor return via your **Period Returns metagroup** (this exists in your setup)
active_factor_return_ccy, frc_map = None, None
for mg in [f"{RM}.Period Returns", f"{RM}.Factor Returns", f"{RM}.Returns"]:
    try:
        frc_map = _try_call("compute_factor_return_contributions", [
            dict(risk_model=RM, metagroup=mg, holdings=ah),
        ])
        if isinstance(frc_map, dict) and frc_map:
            active_factor_return_ccy = _sum_numeric_leaves(frc_map)
            break
    except Exception:
        continue

# If your build has an "all" method, this is a valid alternative:
if active_factor_return_ccy is None:
    try:
        active_factor_return_ccy = float(_try_call("compute_all_factor_return_contribution", [
            dict(risk_model=RM, metagroup=f"{RM}.Period Returns", holdings=ah),
        ]))
    except Exception:
        active_factor_return_ccy = float("nan")

active_specific_return_ccy = (active_total_return_ccy - active_factor_return_ccy
                              if active_factor_return_ccy == active_factor_return_ccy
                              else float("nan"))

returns_summary = pd.DataFrame(
    {
        "Return (ccy)": [active_total_return_ccy, active_factor_return_ccy, active_specific_return_ccy],
        "Return (%)"  : [_pct_of_ref(active_total_return_ccy), _pct_of_ref(active_factor_return_ccy), _pct_of_ref(active_specific_return_ccy)],
    },
    index=["Active Total Return (daily)", "Active Factor Return (daily)", "Active Specific Return (daily)"],
)

# ---------- Display ----------
print("=== Active Factor Exposures (decimal) ===")
display(ah_exposures.style.format({"Exposure": "{:+.6f}"}))

print("=== Risk Summary (Tracking Error & components) ===")
display(risk_summary.style.format({"Risk (ccy)": "{:,.2f}", "Risk (%)": "{:.3f}%"}))

if factor_var_df is not None:
    print("=== Factor Variance Decomposition (Active) ===")
    display(factor_var_df.head(50).style.format({"Variance (ccy^2)": "{:,.0f}", "% of TE variance": "{:.3f}%"}))

if risk_contrib_df is not None and not risk_contrib_df.empty:
    print("=== Top 25 Assets by % Contribution to Active Risk ===")
    display(risk_contrib_df.head(25).style.format({"% of TE": "{:.3f}%"}))

print("=== Active Return Decomposition (daily) ===")
display(returns_summary.style.format({"Return (ccy)": "{:,.2f}", "Return (%)": "{:,.4f}%"}))

if isinstance(frc_map, dict) and frc_map:
    factor_return_df = pd.DataFrame(
        [(str(k), (_sum_numeric_leaves(v) if isinstance(v, (dict, list, tuple)) else float(v)))
         for k, v in frc_map.items()],
        columns=["Factor Group", "Factor Return (ccy)"]
    ).sort_values("Factor Return (ccy)", ascending=False)
    factor_return_df["Factor Return (%)"] = factor_return_df["Factor Return (ccy)"].apply(_pct_of_ref)
    print("=== Factor Return Contributions (daily) ===")
    display(factor_return_df.head(50).style.format({"Factor Return (ccy)": "{:,.2f}", "Factor Return (%)": "{:,.4f}%"}))


=== Active Factor Exposures (decimal) ===


,Exposure
Factor,
WW51AxiomaSH.Financial Services,+0.099915
WW51AxiomaSH.Specialty Retail,+0.076312
WW51AxiomaSH.Pharmaceuticals,+0.060355
WW51AxiomaSH.Aerospace & Defense,+0.055431
WW51AxiomaSH.IT Services,+0.050386
WW51AxiomaSH.Consumer Staples Distribution & Retail,+0.044437
WW51AxiomaSH.Interactive Media & Services,+0.036551
WW51AxiomaSH.Health Care Providers & Services,+0.020924
WW51AxiomaSH.Software,+0.016708


=== Risk Summary (Tracking Error & components) ===


,Risk (ccy),Risk (%)
Tracking Error (Total),"72,469,919.98",5.493%
Active Factor Risk,"14,792,054.93",1.121%
Active Specific Risk,"70,944,234.53",5.377%


=== Factor Variance Decomposition (Active) ===


...

,% of TE
NVDA,18.714%
AAPL,7.606%
MELI,6.337%
ERJ,6.149%
CEG,5.741%
LLY,4.445%
GOOGL,4.343%
PLTR,4.169%
ILMN,3.081%
AMZN,2.751%


=== Active Return Decomposition (daily) ===


,Return (ccy),Return (%)
Active Total Return (daily),"-1,841,487.65",-0.1396%
Active Factor Return (daily),"-119,489.94",-0.0091%
Active Specific Return (daily),"-1,721,997.71",-0.1305%


=== Factor Return Contributions (daily) ===


,Factor Group,Factor Return (ccy),Factor Return (%)
0,WW51AxiomaSH.Broadline Retail,"555,573.61",0.0421%
160,WW51AxiomaSH.Life Sciences Tools & Services,"461,856.40",0.0350%
97,WW51AxiomaSH.Beverages,"233,209.39",0.0177%
190,WW51AxiomaSH.Health Care Equipment & Supplies,"193,048.34",0.0146%
81,WW51AxiomaSH.Energy Equipment & Services,"189,635.27",0.0144%
178,WW51AxiomaSH.Pharmaceuticals,"183,303.99",0.0139%
87,WW51AxiomaSH.Biotechnology,"66,420.77",0.0050%
85,WW51AxiomaSH.Machinery,"58,149.93",0.0044%
154,WW51AxiomaSH.IT Services,"55,312.42",0.0042%
116,WW51AxiomaSH.Entertainment,"54,497.83",0.0041%


In [ ]:
pd.options.display.float_format = "{:,.6f}".format
risk_summary.to_csv("risk_summary.csv", index=True)
return_summary.to_csv("return_summary.csv", index=True)
risk_contrib.to_csv("risk_contrib.csv", index=True)


In [ ]:
# write workspace to .wsp file and release license token
ws.write(os.getcwd(),file_name=f"workspace_on_{dates[i]}.wsp",save_reference=False)
ws.destroy()